<a href="https://colab.research.google.com/github/judmn/Deploy-NLP-Model-with-FastAPI/blob/main/Cyberview_BPI_%7C_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Brief
This is a survey project conducted for Cyberview Sdn Bhd, a Tech hub developer that oversees Cyberjaya ecosystem, for its annual Brand Perception Index tracking. It seeks the opinion of the general public, as well as Cyberview's stakeholders with a target of **700 Completed Responses (CR)**.

The requirements are as follow:

Online (90%): 630 CR  
Face-to-face (10%): 70 CR

Additionally, the **Online** responses has the following CR requirement:

Online - Public: 480 CR  
Online - Stakeholders: 150 CR  

From the 10% Face-to-face (70 CR):  
  - Working adults: 40 CR
  - MMU students: 30 CR

Also, there's also a location-based requirement, as follows:  

Cyberjaya (30%): 210 CR  
Klang Valley (50%): 350 CR  
JB & Penang (20%): 140 CR  

`Update as at 30/01/2024:`  
It stands at 45 CR for stakeholders. Since it's difficult to achieve the 150 CR target, the client decided to back-up remaining CRs with respondents from Cyberjaya, with the following breakdown:

-  20 working adults  
-  30 students in universities around Cyberjaya

Data processing steps for this can be referred in `Responses from SurveyMonkey GSheet - 2nd Form` section.

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from typing import Optional

pd.set_option('display.max_rows', None)

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import google sheet using its url and convert it into DataFrame
def import_gsheet(url: str, sheet: str = 'Sheet1') -> pd.DataFrame:
  """
  Import data from a specific GSheet worksheet
  """

  gsheet = gc.open_by_url(url).worksheet(sheet)
  gsheet_values = gsheet.get_all_values()

  # Convert list into dataframe
  df = pd.DataFrame(gsheet_values[1:], columns=gsheet_values[0])

  # Replace blank cells as np.nan, else they won't be recognized as null values
  df = df.replace(['', 'null'], [np.nan, np.nan])

  return df

# Responses from SurveyMonkey GSheet - 1st Form

Things to note:  
The responses are imported from SurveyMonkey platform into Google Sheet via SurveyMonkey extension. However, the answers to the question `Which of the following do you think is the Cyberview logo?`, which uses logo images, are not imported properly by the extension. The only way to process the results is to download them separately from SurveyMonkey and upload it here for concatenation.

## Import data

In [3]:
raw = import_gsheet(url = 'https://docs.google.com/spreadsheets/d/1dXu_pKaVhWYPlGdjv79faEgmx_ptcSYKQGjVWcd788w/',
                    sheet='Cyberview Brand Perception Index Survey 2024')

raw['Response ID'] = raw['Response ID'].astype('int64')

raw.set_index('Response ID', inplace=True)

# Keep only completed responses, CR
cr = raw[raw['Response Status'] == 'completed']

# Decode Collector ID for source tracking
cr['Collector ID'] = cr['Collector ID'].replace(['454722632', '454701149', '454722561'], ['F2F', 'Online', 'Stakeholders'])

cr.head(3)

,Date Created,Date Modified,IP Address,Duration (sec),Response Status,Email,First Name,Last Name,Collector ID,Collector Type,...,Cyberview menyediakan perkhidmatan pelanggan yang cekap dan pantas.,Cyberview mempromosikan dan menyebarkan maklumat tentang Cyberjaya dan tawaran Cyberview dengan berkesan.,Cyberview adalah organisasi pilihan saya untuk mencapai matlamat dan keperluan saya.,Cyberview ialah organisasi pilihan saya untuk bekerjasama/berkolaborasi,Saya akan mengesyorkan Cyberview kepada rakan dan keluarga saya.,"Cyberview sangat responsif, cekap, berorientasikan hasil dan membawa perubahan.","Cyberview menunjukkan nilai teguh iaitu jujur, bersemangat dan menunjukkan integriti yang tinggi.",Cyberview terbuka kepada kerjasama dalam memperkasakan pelanggan dan komunitinya.,Cyberview konsisten dan mempunyai rasa tanggungjawab yang tinggi.,Cyberview merupakan organisasi yang proaktif dan mahir dalam merealisasikan sesuatu sehingga mencapai kejayaan.
Response ID,,,,,,,,,,,,,,,,,,,,,
118513780738,2024-01-16 17:22:55,2024-01-16 22:47:11,161.142.155.162,19455,completed,Not Answered,Not Answered,Not Answered,Online,weblink,...,Not Answered,Not Answered,Not Answered,Not Answered,Not Answered,Not Answered,Not Answered,Not Answered,Not Answered,Not Answered
118513799025,2024-01-16 17:58:56,2024-01-21 7:48:30,60.50.200.23,395373,completed,Not Answered,Not Answered,Not Answered,Online,weblink,...,Agree,Neither Agree nor Disagree,Agree,Agree,Agree,Agree,Agree,Agree,Agree,Agree
118513808111,2024-01-16 18:17:23,2024-01-16 22:45:43,60.50.200.23,16100,completed,Not Answered,Not Answered,Not Answered,Online,weblink,...,Agree,Neither Agree nor Disagree,Agree,Agree,Agree,Agree,Agree,Agree,Agree,Agree


In [4]:
'''
Split responses between English & Malay version for Malay -> English translation

There are some invalid responses where respondent answered both English & Malay versions, which isn't supposed to happen.
Therefore, the conditions for this filtering has to include checking the other version to have `Not Answered` value in the first question.
'''

# English responses
cr_eng = cr.loc[(cr['Choose your preferred language to answer this survey.'] == 'English') & (cr['Lokasi pekerjaan (jika masih bekerja) / Lokasi tempat tinggal (jika tidak bekerja)'] == 'Not Answered'), 'IP Address':'Cyberview is a go-getter organisation and makes things happen.']

# BM responses
cr_bm = pd.concat([
    cr.loc[(cr['Choose your preferred language to answer this survey.'] == 'Bahasa Melayu') & (cr['Work location (if you\'re in active employment) / Residential location (if you\'re not in the labor force)'] == 'Not Answered'), 'IP Address':'Choose your preferred language to answer this survey.'],
    cr.loc[(cr['Choose your preferred language to answer this survey.'] == 'Bahasa Melayu') & (cr['Work location (if you\'re in active employment) / Residential location (if you\'re not in the labor force)'] == 'Not Answered'), 'Lokasi pekerjaan (jika masih bekerja) / Lokasi tempat tinggal (jika tidak bekerja)':]
    ], axis='columns')

In [5]:
# Confirm that both English & BM versions has the same no. of columns
print(cr_eng.shape[1] == cr_bm.shape[1])

True


In [6]:
# Convert BM questionnaire into English
cr_bm.columns = cr_eng.columns # Both versions have the same questionnaire sequence

In [7]:
# Combine both dataframes
cr_all = pd.concat([cr_eng, cr_bm], axis='index')

In [8]:
# Keep required columns only
ip_address = cr_all[['IP Address']]

collector_id = cr_all[['Collector ID']]

phone_no = cr_all[['Phone No. (for e-wallet token distribution) [format: 60123456789]']]

questions = cr_all.loc[:, 'Work location (if you\'re in active employment) / Residential location (if you\'re not in the labor force)':]

cr_all_trim = pd.concat([ip_address, collector_id, phone_no, questions], axis='columns')

In [9]:
# Drop the question on logo recognition
cr_all_trim.drop(columns=['Which of the following do you think is the Cyberview logo?'], inplace=True)

## Translate responses in BM to English

In [10]:
# Translation
translate = {'Selangor (selain Cyberjaya)': 'Selangor (other than Cyberjaya)',
             'Lelaki': 'Male',
             'Perempuan': 'Female',
             'Melayu': 'Malay',
             'Cina': 'Chinese',
             'India': 'Indian',
             '57 dan ke atas': '57 and older',
             'Pelajar': 'Student',
             'Profesional': 'Professionals',
             'Pekerja kolar putih': 'White collar',
             'Pekerja kolar biru': 'Blue collar',
             'Bekerja sendiri': 'Self-employed',
             'Pemilik perniagaan': 'Business owners',
             'Pesara': 'Retiree',
             'Lain-lain': 'Others',
             'Syarikat berasaskan teknologi (hampir keseluruhan)':'Mostly tech-based company',
             'Syarikat berasaskan teknologi (separa)': 'Partially tech-based company',
             'Syarikat tidak berasaskan teknologi':'Non tech-based company',
             'A) Kurang daripada 50':'A) Less than 50',
             'F) 1001 dan ke atas':'F) 1001 and more',
             'Pengurusan':'Management',
             'Pengurusan pertengahan (middle management)':'Middle Management',
             'Pemantauan / Ketua Pasukan':'Supervisory / Team Leader',
             'Sokongan dan penyelarasan (Eksekutif dan ke bawah)':'Support and Coordination (Executive and lower)',
             'Saya tidak mempunyai sebarang hubungan dengan Cyberview':'I don’t have any relationship with Cyberview',
             'Penyewa atau pelanggan Cyberview':'Cyberview\'s client / tenant',
             'Pembekal perkhidmatan':'Service provider',
             'Rakan kongsi program teknologi':'Tech Programme partner',
             'Rakan kongsi perniagaan':'Business partner',
             'Saya tahu tentang Cyberview tetapi kurang pasti akan apa yang mereka lakukan':'I know about Cyberview but not sure what Cyberview does',
             'Saya tidak pernah mendengar / tidak tahu tentang Cyberview sebelum menjawab kaji selidik ini':'I have not heard of / did not know about Cyberview before this survey',
             'Saya tahu tentang Cyberview dan faham akan jenis perniagaan mereka':'I know about Cyberview and understand their nature of business',
             'Pemaju hab teknologi yang memantau ekosistem Cyberjaya':'Tech hub developer that oversees Cyberjaya ecosystem',
             'Pemaju utama di Cyberjaya':'A master developer for Cyberjaya',
             'Pengurus bandar siber':'A cybercity manager',
             'Pemaju hartanah':'A property developer',
             'Pemilik tanah Cyberjaya':'The landowner of Cyberjaya',
             'Tiada di atas / Saya kurang pasti':'None of the above / I’m not sure'
             }

In [11]:
# Apply translation
cr_all_trim.replace(translate, inplace=True)

In [12]:
# cr_all_trim['Race'].unique()

In [13]:
# Filter out non-Malaysian
cr_all_trim = cr_all_trim.loc[cr_all_trim['Race'].isin(['Malay', 'Chinese', 'Indian', 'Bumiputera Sabah/Sarawak', 'Orang Asal', 'Punjabi', 'Sikh'])]

In [14]:
# Sanity check on all values
for col in cr_all_trim.columns:
  if col not in ['Response ID', 'IP Address', 'Phone No. (for e-wallet token distribution) [format: 60123456789]']:
    print(cr_all_trim[col].value_counts(normalize=False))
    print('\n')

Online          796
F2F              82
Stakeholders     47
Name: Collector ID, dtype: int64


Cyberjaya                          386
Selangor (other than Cyberjaya)    169
Kuala Lumpur                       149
Pulau Pinang                       104
Johor Bahru                         60
Putrajaya                           57
Name: Work location (if you're in active employment) / Residential location (if you're not in the labor force), dtype: int64


Male      465
Female    460
Name: Gender, dtype: int64


Malay                       580
Chinese                     232
Indian                       92
Bumiputera Sabah/Sarawak     17
Sikh                          2
Punjabi                       1
Orang Asal                    1
Name: Race, dtype: int64


25-40           519
41-56           270
57 and older     93
18-24            43
Name: Age range, dtype: int64


Professionals      397
White collar       157
Self-employed       97
Others              90
Student             54
Retiree  

# Responses from SurveyMonkey GSheet - 2nd Form

## Import data

In [15]:
raw_student = import_gsheet(url = 'https://docs.google.com/spreadsheets/d/1dXu_pKaVhWYPlGdjv79faEgmx_ptcSYKQGjVWcd788w/',
                            sheet='Cyberview Brand Perception Index Survey 2024 (Students)')

raw_student.set_index('Response ID', inplace=True)

# Keep only completed responses, CR
cr_student = raw_student.loc[raw_student['Response Status'] == 'completed']

# Decode Collector ID respectively
cr_student['Collector ID'] = cr_student['Collector ID'].replace(['429889948'], ['Online - Student'])

# cr_student.head(3)

In [16]:
# cr_student['Which university you are currently studying at?'].value_counts(normalize=True)

In [17]:
# Drop questions that don't exist in the 1st Form
cr_student.drop(columns=['Which university you are currently studying at?',
                         'Industry – What is the nature of your study?',
                         'Which of the following do you think is the Cyberview logo?'],
                inplace=True)

## Manully insert columns with default values

In [18]:
# Manually insert `Age range` column
cr_student['Age range'] = '18-24'

# Manually insert `Occupation - What is your profession?` column
cr_student['Occupation - What is your profession?'] = 'Student'

# Manually insert location
cr_student['Work location (if you\'re in active employment) / Residential location (if you\'re not in the labor force)'] = 'Cyberjaya'

In [19]:
# Keep required columns only
ip_address_2 = cr_student[['IP Address']]

collector_id_2 = cr_student[['Collector ID']]

questions_2 = cr_student.loc[:,'Phone No. (for e-wallet token distribution) [format: 60123456789]':]

cr_student_trim = pd.concat([ip_address_2, collector_id_2, questions_2], axis='columns')

In [20]:
# Filter out non-Malaysian
cr_student_trim = cr_student_trim.loc[cr_student_trim['Race'].isin(['Malay', 'Chinese', 'Indian', 'Bumiputera Sabah/Sarawak', 'Orang Asal', 'Punjabi', 'Sikh'])]

In [21]:
cr_student_trim.head(3)

,IP Address,Collector ID,Phone No. (for e-wallet token distribution) [format: 60123456789],Gender,Race,Household monthly income category,Relation with Cyberview: What is your relationship with Cyberview?,Which of the following best describes your impression of Cyberview?,Which of the following do you think is the primary role of Cyberview?,Where have you seen or heard about Cyberview? | Online search,...,Cyberview is my preferred organisation to work with/collaborate with,I would recommend Cyberview to my friends and families.,"Cyberview is responsive, alert, is result-oriented and creates change.","Cyberview demonstrates strong values of being honest, passionate and demonstrates high integrity.",Cyberview is open to collaboration to empower its clients and community.,Cyberview is consistent and has a strong sense of accountability.,Cyberview is a go-getter organisation and makes things happen.,Age range,Occupation - What is your profession?,Work location (if you're in active employment) / Residential location (if you're not in the labor force)
Response ID,,,,,,,,,,,,,,,,,,,,,
114521482199,161.142.154.164,Online - Student,60129823856,Male,Malay,"A) Less than RM3,440",I don’t have any relationship with Cyberview,I know about Cyberview but not sure what Cyber...,A cybercity manager,FALSE,...,Strongly Agree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,18-24,Student,Cyberjaya
114521484514,202.184.154.31,Online - Student,601137091353,Male,Chinese,"B) RM3,440 - RM5,249",I don’t have any relationship with Cyberview,I have not heard of / did not know about Cyber...,A cybercity manager,FALSE,...,Neither Agree nor Disagree,Agree,Agree,Neither Agree nor Disagree,Agree,Agree,Agree,18-24,Student,Cyberjaya
114521487087,42.153.128.121,Online - Student,601163193829,Female,Malay,"A) Less than RM3,440",I don’t have any relationship with Cyberview,I know about Cyberview but not sure what Cyber...,None of the above / I’m not sure,FALSE,...,Agree,Strongly Agree,Agree,Agree,Strongly Agree,Agree,Agree,18-24,Student,Cyberjaya


# Combine both forms

In [22]:
cr_combined = pd.concat([cr_all_trim, cr_student_trim], axis='index', sort=False)

print(cr_combined.shape)

(971, 59)


In [23]:
for col in cr_combined.columns:
  if col not in ['Response ID', 'IP Address', 'Phone No. (for e-wallet token distribution) [format: 60123456789]']:
    print(cr_combined[col].value_counts())
    print('\n')

Online              796
F2F                  82
Stakeholders         47
Online - Student     46
Name: Collector ID, dtype: int64


Cyberjaya                          432
Selangor (other than Cyberjaya)    169
Kuala Lumpur                       149
Pulau Pinang                       104
Johor Bahru                         60
Putrajaya                           57
Name: Work location (if you're in active employment) / Residential location (if you're not in the labor force), dtype: int64


Female    489
Male      482
Name: Gender, dtype: int64


Malay                       607
Chinese                     243
Indian                      100
Bumiputera Sabah/Sarawak     17
Sikh                          2
Punjabi                       1
Orang Asal                    1
Name: Race, dtype: int64


25-40           519
41-56           270
57 and older     93
18-24            89
Name: Age range, dtype: int64


Professionals      397
White collar       157
Student            100
Self-employed      

# Remove duplicates

In [24]:
# Since only a single TnG e-wallet will be given to each unique phone no., we need to ensure there's no phone no. duplicates
cr_combined.drop_duplicates(subset='Phone No. (for e-wallet token distribution) [format: 60123456789]', inplace=True)

print(f'CR after removing duplicates: {cr_combined.shape[0]}')

CR after removing duplicates: 966


In [25]:
ip_duplicate = cr_combined.loc[cr_combined.duplicated(subset='IP Address', keep=False)].sort_values(by='IP Address')

ip_duplicate['IP Address'].value_counts()

115.135.28.103     7
103.61.127.165     5
103.253.105.52     4
49.124.200.218     3
183.171.175.39     3
202.187.67.197     3
45.121.38.30       3
175.142.80.150     3
113.210.90.213     3
183.171.175.119    2
45.121.38.28       2
45.121.38.26       2
27.125.248.68      2
219.92.18.205      2
58.71.165.6        2
183.171.191.136    2
60.48.168.110      2
103.253.105.16     2
161.142.156.67     2
172.225.181.81     2
103.253.105.44     2
161.142.154.184    2
161.142.154.169    2
14.192.196.43      2
120.138.95.7       2
120.138.95.0       2
113.211.210.249    2
113.211.110.118    2
113.210.50.104     2
110.159.85.172     2
60.53.34.8         2
Name: IP Address, dtype: int64

# CR Monitoring

## Location-based CR

In [26]:
cr_all_trim.loc[cr_all_trim['Occupation - What is your profession?'] != 'Student', 'Household monthly income category'].value_counts()

B) RM3,440 - RM5,249     218
D) RM7,690 - RM11,819    179
C) RM5,250 - RM7,689     175
A) Less than RM3,440     158
E) RM11,820 and above    141
Name: Household monthly income category, dtype: int64

In [27]:
# If CR from working adults who are 'A) Less than RM3,440' in household income are removed
cr_all_rich = cr_all_trim.loc[(cr_all_trim['Occupation - What is your profession?'] != 'Student') & (cr_all_trim['Household monthly income category'] != 'A) Less than RM3,440')]

In [28]:
cr_all_rich['Work location (if you\'re in active employment) / Residential location (if you\'re not in the labor force)'].value_counts()

Cyberjaya                          294
Kuala Lumpur                       129
Selangor (other than Cyberjaya)    129
Pulau Pinang                        68
Putrajaya                           49
Johor Bahru                         44
Name: Work location (if you're in active employment) / Residential location (if you're not in the labor force), dtype: int64

## F2F - Working adults

In [29]:
f2f = cr_all_trim.loc[(cr_all_trim['Collector ID'] == 'F2F') & (cr_all_trim['Occupation - What is your profession?'] != 'Student')]

print(f'CR: {f2f.shape[0]}')

CR: 43


## F2F - Students

In [30]:
f2f_student = cr_combined.loc[(cr_combined['Collector ID'] == 'F2F') & (cr_combined['Occupation - What is your profession?'] == 'Student')]

print(f'CR: {f2f_student.shape[0]}')

CR: 39


## Online
CR requirement for online respondent: 480

In [31]:
online = cr_all_trim.loc[cr_all_trim['Collector ID'] == 'Online']

print(f'CR: {online.shape[0]}')

CR: 796


In [32]:
online['Work location (if you\'re in active employment) / Residential location (if you\'re not in the labor force)'].value_counts()

Cyberjaya                          293
Selangor (other than Cyberjaya)    154
Kuala Lumpur                       133
Pulau Pinang                       104
Johor Bahru                         60
Putrajaya                           52
Name: Work location (if you're in active employment) / Residential location (if you're not in the labor force), dtype: int64

In [33]:
pd.pivot_table(data = online,
               index='Work location (if you\'re in active employment) / Residential location (if you\'re not in the labor force)',
               columns='Occupation - What is your profession?',
               values='Collector ID',
               aggfunc='count')

Occupation - What is your profession?,Blue collar,Business owners,Others,Professionals,Retiree,Self-employed,Student,White collar
Work location (if you're in active employment) / Residential location (if you're not in the labor force),,,,,,,,
Cyberjaya,5.0,16.0,35.0,143.0,6.0,31.0,9.0,48.0
Johor Bahru,3.0,4.0,9.0,17.0,6.0,10.0,1.0,10.0
Kuala Lumpur,8.0,4.0,7.0,66.0,6.0,10.0,NaN,32.0
Pulau Pinang,3.0,4.0,13.0,35.0,14.0,19.0,1.0,15.0
Putrajaya,2.0,NaN,6.0,31.0,NaN,1.0,2.0,10.0
Selangor (other than Cyberjaya),4.0,4.0,10.0,62.0,21.0,21.0,2.0,30.0


## Stakeholders Only
CR requirement for stakeholders: 150

In [34]:
stakeholders = cr_all_trim.loc[cr_all_trim['Collector ID'] == 'Stakeholders']

print(f'CR: {stakeholders.shape[0]}')

CR: 47


In [35]:
# for col in stakeholders.columns:
#   if col not in ['IP Address', 'Phone No. (for e-wallet token distribution) [format: 60123456789]']:
#     print(stakeholders[col].value_counts())
#     print("\n")

# Manual Upload of Logo Recognition Responses
The responses were downloaded from SurveyMonkey as `xls` format with the following filters:

1.  Completed responses only
2.  For question, only the question on logo recognition was chosen

In [37]:
import time

file_name = [
    'Cyberview_logo_060224.xlsx',
    'Cyberview_student_logo_060224.xlsx'
    ]

logo_df = []

for f in file_name:

  print(f'Processing {f} ..')

  logo = pd.read_excel(f)

  logo.rename(columns={'Respondent ID':'Response ID'}, inplace=True)

  logo['Response ID'] = logo['Response ID'].astype('int64')

  logo.set_index('Response ID', inplace=True)

  # Conditional processing for form with both BM & English versions
  if 'Antara berikut, yang manakah merupakan logo Cyberview?' in logo.columns:

    logo_eng = logo.loc[(logo['Which of the following do you think is the Cyberview logo?'].notnull()) & (logo['Antara berikut, yang manakah merupakan logo Cyberview?'].isnull()), ['Which of the following do you think is the Cyberview logo?']]

    logo_bm = logo.loc[(logo['Antara berikut, yang manakah merupakan logo Cyberview?'].notnull()) & (logo['Which of the following do you think is the Cyberview logo?'].isnull()), ['Antara berikut, yang manakah merupakan logo Cyberview?']]

    logo_bm.rename(columns={'Antara berikut, yang manakah merupakan logo Cyberview?':'Which of the following do you think is the Cyberview logo?'},
                  inplace=True)

    logo_bm.replace({'Tidak pasti': 'I\'m not sure'}, inplace=True)

    logo_all = pd.concat([logo_eng, logo_bm], axis='index', verify_integrity=True)

  else:
    logo_all = logo

  logo_all = logo_all[['Which of the following do you think is the Cyberview logo?']]

  logo_df.append(logo_all)

  print(f'Processing completed. ', end='')
  print(f'DF shape: {logo_all.shape}')
  time.sleep(1)

logo_combined = pd.concat(logo_df)

logo_combined.replace({'Image 1': 'Image 1 (MSC logo)',
                       'Image 2': 'Image 2 (Cyberview logo)',
                       'Image 3': 'Image 3 (Cyberjaya logo)',
                       'Image 4': 'Image 4 (Cyberview Resort & Spa)'
                       },
                      inplace=True)

print(display(logo_combined.head()))
print(logo_combined.value_counts())

Processing Cyberview_logo_060224.xlsx ..
Processing completed. DF shape: (944, 1)
Processing Cyberview_student_logo_060224.xlsx ..
Processing completed. DF shape: (50, 1)


,Which of the following do you think is the Cyberview logo?
Response ID,
118529091140,Image 2 (Cyberview logo)
118528810037,Image 2 (Cyberview logo)
118528730845,Image 2 (Cyberview logo)
118528708168,Image 2 (Cyberview logo)
118528693079,Image 2 (Cyberview logo)


None
Which of the following do you think is the Cyberview logo?
Image 2 (Cyberview logo)                                      643
I'm not sure                                                  152
Image 4 (Cyberview Resort & Spa)                              114
Image 1 (MSC logo)                                             60
Image 3 (Cyberjaya logo)                                       25
dtype: int64


## Join Logo Recognition question with main DF

In [ ]:
# cr_combined.loc['118528453547']

In [38]:
# cr_merged = pd.merge(cr_combined, logo_combined, left_index=True, right_index=True)

# cr_merged.info()

# Brand Perception Index Calculation
There's an Excel template given by the client for the BPI calculation. All we need is to input the count of each relevant responses accordingly, and the BPI and its sub-components (ie. Brand Awareness, Brand Association, etc.) will be calculated automatically.

## Brand Awareness (Cyberview Logo)

In [39]:
logo_combined[['Which of the following do you think is the Cyberview logo?']].value_counts().to_frame(name='count').reset_index()

,Which of the following do you think is the Cyberview logo?,count
0,Image 2 (Cyberview logo),643
1,I'm not sure,152
2,Image 4 (Cyberview Resort & Spa),114
3,Image 1 (MSC logo),60
4,Image 3 (Cyberjaya logo),25


## Primary Role

In [45]:
primary_role = cr_combined[['Which of the following do you think is the primary role of Cyberview?']].value_counts().to_frame(name='count').reset_index()

primary_role

,Which of the following do you think is the primary role of Cyberview?,count
0,Tech hub developer that oversees Cyberjaya eco...,438
1,A master developer for Cyberjaya,205
2,None of the above / I’m not sure,149
3,A property developer,79
4,A cybercity manager,57
5,The landowner of Cyberjaya,38


## Brand Association

In [46]:
brand_assoc = [
    'Cyberview develops and is in-charge of the Cyberjaya new masterplan.',
    'Cyberview plays a pivotal role in bringing tech investments into Cyberjaya.',
    'Cyberview forges strong partnerships with tech ecosystem partners to strengthen Cyberjaya’s position as a Global Technology Hub.',
    'Cyberview provides a one-stop centre for investment facilitation through Cyberjaya Investment and Services Centre.',
    'Cyberview leads the transformation of Cyberjaya into a Global Technology Hub.',
    'Cyberview enables Cyberjaya as a smart city and living lab to pilot innovations.',
    'Cyberview plays a key role in providing development programmes for startups and tech companies.',
    'Cyberview forges strong partnerships with startups and tech community through collaborative efforts to take them to the next level.'
    ]

for q in brand_assoc:
  print(cr_combined[[q]].value_counts().to_frame(name='count').reset_index())
  print('\n')

  Cyberview develops and is in-charge of the Cyberjaya new masterplan.  count
0                                              Agree                      464
1                         Neither Agree nor Disagree                      286
2                                     Strongly Agree                      178
3                                  Strongly Disagree                       27
4                                           Disagree                       11


  Cyberview plays a pivotal role in bringing tech investments into Cyberjaya.  \
0                                              Agree                            
1                         Neither Agree nor Disagree                            
2                                     Strongly Agree                            
3                                  Strongly Disagree                            
4                                           Disagree                            

   count  
0    522  
1    241  
2    167  

## Perceived Quality

In [47]:
quality = [
    'Cyberview acts as a key player in attracting and growing tech investments in Cyberjaya.',
    'Cyberview is proactive in strengthening Cyberjaya’s ecosystem towards elevating Cyberjaya into a Global Tech Hub through its new masterplan, smart city and living lab initiatives.',
    'Cyberview has the capacity and capability to elevate Cyberjaya into a Global Tech Hub.',
    'Cyberview provides effective development programmes for startups and tech companies.',
    'Cyberview provides efficient and fast client services.',
    'Cyberview effectively promotes and disseminates information about Cyberjaya and Cyberview’s offerings.'
    ]

for q in quality:
  print(cr_combined[[q]].value_counts().to_frame(name='count').reset_index())
  print('\n')

  Cyberview acts as a key player in attracting and growing tech investments in Cyberjaya.  \
0                                              Agree                                        
1                         Neither Agree nor Disagree                                        
2                                     Strongly Agree                                        
3                                           Disagree                                        
4                                  Strongly Disagree                                        

   count  
0    526  
1    236  
2    164  
3     20  
4     20  


  Cyberview is proactive in strengthening Cyberjaya’s ecosystem towards elevating Cyberjaya into a Global Tech Hub through its new masterplan, smart city and living lab initiatives.  \
0                                              Agree                                                                                                                                    
1  

## Brand Preference

In [48]:
brand_pref = [
    'Cyberview is my preferred organisation to go to, to achieve my goals and needs.',
    'Cyberview is my preferred organisation to work with/collaborate with',
    'I would recommend Cyberview to my friends and families.'
    ]

for q in brand_pref:
  print(cr_combined[[q]].value_counts().to_frame(name='count').reset_index())
  print('\n')

  Cyberview is my preferred organisation to go to, to achieve my goals and needs.  \
0                         Neither Agree nor Disagree                                
1                                              Agree                                
2                                     Strongly Agree                                
3                                           Disagree                                
4                                  Strongly Disagree                                

   count  
0    439  
1    359  
2    112  
3     34  
4     22  


  Cyberview is my preferred organisation to work with/collaborate with  count
0                                              Agree                      395
1                         Neither Agree nor Disagree                      393
2                                     Strongly Agree                      126
3                                           Disagree                       32
4                              

## Brand Personality

In [49]:
brand_persona = [
    'Cyberview is responsive, alert, is result-oriented and creates change.',
    'Cyberview demonstrates strong values of being honest, passionate and demonstrates high integrity.',
    'Cyberview is open to collaboration to empower its clients and community.',
    'Cyberview is consistent and has a strong sense of accountability.',
    'Cyberview is a go-getter organisation and makes things happen.'
]

for q in brand_persona:
  print(cr_combined[[q]].value_counts().to_frame(name='count').reset_index())
  print('\n')

  Cyberview is responsive, alert, is result-oriented and creates change.  \
0                                              Agree                       
1                         Neither Agree nor Disagree                       
2                                     Strongly Agree                       
3                                           Disagree                       
4                                  Strongly Disagree                       

   count  
0    413  
1    390  
2    114  
3     26  
4     23  


  Cyberview demonstrates strong values of being honest, passionate and demonstrates high integrity.  \
0                                              Agree                                                  
1                         Neither Agree nor Disagree                                                  
2                                     Strongly Agree                                                  
3                                  Strongly Disagree           